In [ ]:
from sklearn import svm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Copy of q3b: Evaluation metric

def eval_metric(true_labels, predicted_labels):
    """
    use classification accuracy, or the percent of examples classified correctly,
    as a measure of the classifier performance.
    
    Args:
        true_labels: the set of true labels of the dataset
        predicted_labels: the set of labels predicted by the model
    
    Returns:
        the (unweighted) accuracy score
    """
    assert len(true_labels) == len(predicted_labels)
    total_num = len(true_labels)
    accurate_num = 0
    for i in range(total_num):
        if true_labels[i] == predicted_labels[i]:
            accurate_num += 1
    
    return accurate_num / total_num

In [ ]:
# Train a linear SVM on the MNIST dataset with a given hyperparameter C

def mnist_linear_svm(train_data_subset, train_labels_subset, hyper_c):
    # use linear SVM to train the model
    model = svm.SVC(kernel="linear", max_iter=10000, C=hyper_c)
    model.fit(train_data_subset, train_labels_subset)
    
    return model

In [ ]:
mnist_data = np.load("data/mnist-data.npz")
fields = "test_data", "training_data", "training_labels"
mnist_train_val_data = mnist_data[fields[1]]
mnist_train_val_labels = mnist_data[fields[2]]
# For the MNIST dataset, write code that sets aside 10,000 training images as a validation set.
mnist_val_data_num = 10000
train_set_size = 12000 # use 12,000 samples as train subset
mnist_indices = np.random.permutation(len(mnist_train_val_data))
mnist_val_data = mnist_train_val_data[mnist_indices[:mnist_val_data_num]]
mnist_val_labels = mnist_train_val_labels[mnist_indices[:mnist_val_data_num]]
mnist_train_data = mnist_train_val_data[mnist_indices[mnist_val_data_num:]]
mnist_train_labels = mnist_train_val_labels[mnist_indices[mnist_val_data_num:]]
# train the model with 0~train_set_size subset of all the train data
mnist_train_data_subset = mnist_train_data[:train_set_size]
mnist_train_labels_subset = mnist_train_labels[:train_set_size]

# In MNIST, our feature vector for an image will be a row vector with all the pixel values
# concatenated in a row major (or column major) order
flattened_mnist_train_data_subset = mnist_train_data_subset.reshape((train_set_size, -1))
flattened_mnist_val_data = mnist_val_data.reshape((mnist_val_data_num, -1))

mnist_hyper_C = np.array((1.e-10, 1.e-9, 1.e-8, 1.e-7, 1.e-6, 1.e-5, 1.e-4, 1.e-3, 1.e-2, 1.e-1, 1, 10))
mnist_training_accuracies = np.zeros(len(mnist_hyper_C))
mnist_validation_accuracies = np.zeros(len(mnist_hyper_C))
mnist_index = 0
for hyper_c in mnist_hyper_C:
    model = mnist_linear_svm(flattened_mnist_train_data_subset, mnist_train_labels_subset, hyper_c)
    pred_mnist_val_labels = model.predict(flattened_mnist_val_data)
    pred_mnist_train_labels_subset = model.predict(flattened_mnist_train_data_subset)
    mnist_training_accuracies[mnist_index] = \
        eval_metric(mnist_train_labels_subset, pred_mnist_train_labels_subset)
    mnist_validation_accuracies[mnist_index] = \
        eval_metric(mnist_val_labels, pred_mnist_val_labels)
    mnist_index += 1

# Plot the accuracies
plt.plot(mnist_hyper_C, mnist_training_accuracies, label="MNIST training accuracy") 
plt.plot(mnist_hyper_C, mnist_validation_accuracies, label="MNIST validation accuracy")
plt.title("MNIST-linearSVM Accuracy vs Hyperparameter C")
plt.xlabel("hyper-C")
plt.xscale("log")
plt.ylabel("training and validation accuracy")
plt.legend()
plt.savefig("q5_hyper_tuning.png")